In [ ]:
import os
import pandas as pd

# This file automates the entire pipeline for assertion generation with chatgpt

## Step 1) Get Asserted Code From Github

### Step 1.1) Clean and process the code
### Step 1.2) Extract Ground-Truth Assertions & Relevant Statistics

In [ ]:
query_string="""SELECT f.repo_name, c.content
FROM `bigquery-public-data.github_repos.files` AS f
JOIN `bigquery-public-data.github_repos.contents` AS c
ON f.id = c.id
WHERE
NOT c.binary
AND f.path LIKE '%.py'
AND REGEXP_CONTAINS(c.content, r'(?m)^\s*assert ')
LIMIT """

def get_asserted_code(num=100000, verbose=True):
    query_string += str(num)
    
    if verbose:
        print("*Running Query:")
        print(query_string)
        print()
    client = bq.Client()
    df = (
        client.query(query_string)
        .result()
        .to_dataframe(
            create_bqstorage_client=True,
        )
    )
    
    # handling duplicates
    init_len = len(df)
    df.drop_duplicates(subset=["content"], keep="first", inplace=True)
    if verbose:
        print("Duplicate Ratio = ", (len(df)/init_len))
        
    # derive assertions
    if verbose:
        print("*Extracting Assertions")
    df["assertions"] = df["content"].apply(lambda code: get_assertions(code, True, verbose))
    return df


conditionals = dict([[cond, i] for i, cond in enumerate(["==", "!=", "<=", ">=", "<", ">"])])
compounding_statements = ["and"]  # TODO: properly account for OR
bad_statements = ["or"]  # TODO: check for more than just 1 index
# TODO: experiment with smaller content window for assertions
def get_assertions(func, is_split=True, verbose=True):
    """
    Format: "assert [expression], [return_string]"
    """
    out = []
    lines = [temp.strip() for temp in func.split('\n') if "assert" in temp and bad_statements[0] not in temp]
    ind = 0
    while ind < len(lines):
        data = lines[ind].strip()
        start = data.find('assert')
        if start != -1:
            # account for combination statements
            for statement in compounding_statements:
                add_statement = data.find(statement)
                if add_statement != -1:
                    extra_line = data[add_statement+len(statement):]
                    lines.insert(ind+1, "assert "+extra_line)
                    data = data[:add_statement].strip()
            
            com = data.find(',')   # parsing out return_string
            if com != -1:
                data = data[:com]

            if is_split:
                data = [var.strip() for var in data.split()]
                assert data[0] == "assert", "something was found before the assertion in this line"
                data = data[1:]
                
                condition = True  # assertion [variable] == condition by default
                if data[0] == "not":  # accounting for not
                    condition = False
                    data = data[1:]
                    
                assert len(data) >= 1, "empty assertion found?: " + data
                if len(data) == 1:  # adding == to simlify
                    data = data + ["==", str(condition)]
                
                for i in range(len(data)):
                    if data[i] == "is":  # simplifying is to ==
                        data[i] = "=="
                    if data[i] in conditionals.keys():  # com
                        data = [' '.join(data[:i]), data[i], ' '.join(data[i+1:])]  # conditionals[data[i]]
                        break
            
            if verbose and len(data) != 3:
                print("Weird assertion found:\n", data, '\n', '\n'.join(lines[ind-1:ind+2]))
                print()
#             assert len(data) == 3, "found conditional-less assertion:\n" + str(data) + '\n' + str(lines[ind-1:ind+2])
            else:
                out.append(data)
        ind += 1
    return out



## Step 2) Generate LLM Prompt & Query a GPT

In [ ]:
def generate_prompt(asserted_code, verbose=True):
    ...
    
    
banned_vars = ['', '*', 'self']
def get_variables(func, verbose=False):
    out = []
    for line in func.split('\n'):
        line = line.strip()
        if "def " in line:  # add params if its a function
            start = line.find('(')
            end = line.find(')')
            for new_param in line[start+1:end].split(','):
                default = new_param.find("=")
                if default != -1:
                    new_param = new_param[:default]
                new_param = new_param.strip()
                if new_param not in out and new_param not in banned_vars:
                    if verbose:
                        print("*Found  {", new_param, "}  at:\n", line, '\n')
                    out.append(new_param)
        else: # add variables if equals operation
            find_var = line.find(' = ')
            if find_var != -1:
                new_var = line[:find_var].strip()
                
                if ',' in new_var: # handle tuple equalities edge case (ex: a, b, c = fn_output())
                    var_list = [tuple_var.strip() for tuple_var in new_var.split(',')]
                else:
                    var_list = [new_var]
                for new_var in var_list:
                    if new_var not in out and new_var not in banned_vars:
                        if verbose:
                            print("**Found  {", new_var, "}  at:\n", line, '\n')
                        out.append(new_var)
            # TODO: handle indexing
    return out

# out = get_variables(df.sample()["content"].iloc[0])
get_vars = lambda code: get_variables(code)
df["variables"] = df["content"].apply(get_vars)
df

## Step 3) Parse & Evaluate GPT's Response

### Step 3.1) Restore the assertion(s) generated to code and evaluate
> Metrics of evaluation, does it run? does it add to the code? is it ground-truth-like? human evaluator rank? gpt evaluator rank?